In [1]:
import numpy as np
import cv2
from skimage import color

def read_video_cv2(n_frames=10):
    cap = cv2.VideoCapture("./survelliance/09152008flight2tape1_7.mpg")
    all = []
    i = 0
    while cap.isOpened() and i < n_frames:
        ret, frame = cap.read()
        img = color.rgb2gray(frame)
        arr = np.array(img)
        all.append(arr)
        i += 1
    return np.array(all)

video_seg = read_video_cv2()
print(video_seg.shape)

(10, 480, 720)


In [2]:
import scipy

u, d, v = scipy.sparse.linalg.svds(video_seg[0], k=3)
print(d.shape)

(3,)


In [3]:
# the shape of B
B0 = np.matmul(u, v)
B0.shape

(480, 720)

In [4]:
# I denotes the jth frame in sequence, where each I is a column vector with m pixels
m = 480
n = 720
p1 = 6
p2 = 8
D = np.ones((m, n))
B = np.ones((m, n))
S = np.ones((m, n))
tau = np.ones((1, p1))
each_I = np.transpose(np.ones((1, m)))
B_hat = np.ones((m, n))
S_hat = np.ones((1, n))


def c_subsolution(D, B_hat):
    c_sum = 0
    for i in range(0, D.shape[0]):
        for j in range(0, D.shape[1]):
            c_sum += 0.5 *((D[i][j] - B_hat[i][j]) ** 2)
    return c_sum

c_subsolution(D, B_hat)

0.0

In [5]:
def B_hat_estimate(D, B_hat, alpha):
    B_hat = singular_threshold(com_projection(I, D, S_hat) + orthogonal_projection(B_hat, S_hat), alpha)
    return B_hat

In [6]:
I = np.identity(m)

# complementary projector 
# x = complementary(X, S) + orthogonal_projection(X, S)
def com_projection(identity, input_matrix, subspace):
    P = np.matmul(input_matrix, np.transpose(input_matrix))
    print(P.shape)
    return identity - P

com_pro = com_projection(I, D, S)

(480, 480)


In [7]:
n = 720
S_hat = np.ones((1, 720))
torch.tensor((np.dot(B[4].reshape((1, 720)), np.transpose(S_hat))/np.tensordot(S_hat, S_hat)) * S_hat).shape

NameError: name 'torch' is not defined

In [153]:
# it represents the orthogonal projection of matrix X onto linear space of matrices supported by S_hat
# S_hat is the subspace

B = np.ones((480, 720))
S_hat = np.ones((1, 720))

def orthogonal_projection(B, S_hat):
    # each row in B is a vector 
    vector_space = []
    print(len(B))
    for i in range(0, len(B)):
        each_projector = (np.matmul(B[i].reshape((1, 720)), np.transpose(S_hat))/np.tensordot(S_hat, S_hat)) * S_hat
        vector_space.append(each_projector)
    return vector_space

oth_projector = np.array(orthogonal_projection(B, S_hat)).reshape(480, 720)
print(np.array(B-oth_projector).shape)

480
(480, 720)


In [167]:
def orthogonal_projection_try(B, S_hat):
    return (np.tensordot(B, S_hat)/np.tensordot(S_hat, S_hat)) * S_hat

B = np.ones((480, 720))
S_hat = np.ones((480, 720))
print(orthogonal_projection_try(B, S_hat).shape)

(480, 720)


In [156]:
np.array(orthogonal_projection(D, S_hat)).shape

480


(480, 1, 720)

In [ ]:
np.dot(B[0].reshape(1, 720), np.transpose(S_hat))/np.tensordot(S_hat, S_hat)

In [ ]:
import torch

# shape of each_D is m * 1 
# shape of tau is 1 * p1
# the shape of B_hat is m * p1
each_D = np.ones((m, 1))
each_tau = np.ones((1, p1))
def cal_B_hat(each_D, each_tau):
    B_hat = []
    for i in range(0, n):
        each_column = np.matmul(each_D, each_tau)
        B_hat.append(each_column)
    return B_hat
    
res = cal_B_hat(each_D, each_tau)
print(torch.tensor(res).shape)    

In [ ]:
# weighted least squares 
import torch
from torch.autograd.functional import jacobian
import tensorflow as tf

rank = 3

tau = np.ones((p1, 30))

# the first parameter is the shape of B_hat m * p 
# the second paramter is the shape of tau 
def wls_subproblem(com_projector, input_tau):
    input_weighted = tf.norm(com_projector, ord=2) ** 2
    param = input_tau
    return torch.linalg.lstsq(torch.tensor(input_weighted), torch.tensor(param), rcond=None, driver='gels')

In [119]:
import numpy as np
import torch
from torch.autograd.functional import jacobian

# manipulate tau onto D, the result is still 480 * 720
tau_hat = np.linalg.norm(singular_threshold(D_parameterized - B_hat, rank, alpha), ord='fro') ** 2

print(tau_hat)

def relation_exp(tau):
    return tau * 2 + tau * tau

def cal_jacobian(inputs):
    return jacobian(relation_exp,  torch.tensor(inputs), create_graph=True)

cal_jacobian(tau_hat)

345482.45449234475


tensor(690966.9090, dtype=torch.float64, grad_fn=<ViewBackward>)

In [121]:
com_projection(I, D_parameterized, S_hat)

(480, 480)


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [18]:
def estimate_delta(D, B_hat):
    input_var = np.subtract(D, B_hat)
    squared_delta = np.var(input_var)
    return squared_delta

estimate_delta(D, B_hat)

0.029514034008903386

In [19]:
def convert_diagonal_alpha(input_dia, rank, alpha):
    res = np.zeros((rank, rank))
    for i in range(0, rank):
        for j in range(0, rank):
            if i == j: 
                res[i][j] = input_dia[i] - alpha
    return res

convert_diagonal_alpha(d, 3, 0.5)

array([[ 21.75750116,   0.        ,   0.        ],
       [  0.        ,  30.9118654 ,   0.        ],
       [  0.        ,   0.        , 226.87020831]])

In [20]:
def cal_s_hat(S_hat):
    S_hat = np.zeros((S_hat.shape[0], S_hat.shape[1]))
    
    for i in range(0, S_hat.shape[0]):
        for j in range(0, S_hat.shape[1]):
            if 0.5 * ((D[i][j] - B_hat[i][j]) ** 2) > beta:
                S_hat[i][j] = 1
    
    return S_hat

Sparse_hat = cal_s_hat(S_hat)    
print(Sparse_hat.shape)

(1, 720)


In [21]:
import tensorflow as tf

beta = 2

# A is the adjacency matrix of graph with m * n
m = 480
n = 720
A = np.ones((m, n))
print(S.shape)
def energy_optimization(D, B_hat, S, A):
    energy_sum = 0
    for i in range(0, m):
        for j in range(0, n):
            tmp1 = (D[i][j] - B_hat[i][j])**2
            mid = beta - 0.5 * tmp1
            tmp2 = mid * S[i][j]
            energy_sum += tmp2
            
    C = c_subsolution(D, B_hat)
    print("shape A: " + str(A.shape))
    print("shape S: " + str(S.shape))
    return energy_sum + gamma1 * tf.norm(np.matmul(A, np.vectorize(S)), ord=1) + C 

(480, 720)


In [118]:
import torch

# the operator of each D and each tau is the transformation parameterization
# B is low-rank, E is intensity shift by outliers, epsilon is gaussian noise 
def dot_operator(D, tau_vec):
    new_D = []
    for i in range(0, n):
        new_D_re = []
        D_re = D[:, i].reshape(1, 480)
        # recycle dot product
        for j in range(0, 480, 6):
            param_D = np.multiply(D_re[:, j:j+6], tau_vec)
            new_D_re.append(param_D)
        new_D.append(np.array(new_D_re).flatten())
        D[:, i] = B[:, i] + E[:, i] + epsilon[:, i]
    print(np.array(new_D).shape)
    return np.transpose(new_D)

tau_each = np.ones((1, 6))
D_parameterized = dot_operator(D, tau_each)
print(D_parameterized.shape)

(720, 480)
(480, 720)


In [ ]:
# size of tau is (p * n, 1)
tau_hat = tf.norm(com_projection(I, dot_operator(D, tau) - B_hat, S_hat), ord='euclidean')
print(tau_hat)
K = np.sqrt(n)

In [24]:
gamma1 = beta
gamma2 = 5*beta

In [25]:
def singular_threshold(input_matrix, rank, alpha):
    u, d, voft = scipy.sparse.linalg.svds(input_matrix, k=rank)
    diag_alpha = convert_diagonal_alpha(d, rank, alpha) 
    return np.matmul(np.matmul(u, diag_alpha), voft)


def get_rank_b(B):
    return tf.norm(B, ord='euclidean')

In [26]:
singular_threshold(D, 3, 0.5).shape

(480, 720)

In [27]:
get_rank_b(B)

<tf.Tensor: shape=(), dtype=float64, numpy=0.0>

In [139]:
print((dot_operator(D, tau) - B_hat).shape)
print(tau.shape)      

(720, 480)
(480, 720)
(1, 6)


In [195]:
def get_rank(input_matrix):
    return np.linalg.matrix_rank(input_matrix)

In [196]:
def weight_lease_square()

def wls_subproblem(input_projection, tau_para)
    return tau_para
    
def cal_jacobian(input_matrix)

SyntaxError: invalid syntax (<ipython-input-196-2a502730c665>, line 1)

In [1]:
import scipy

image_input = video_seg[0]
m = video_seg[0].shape[0]
m = 30
n = video_seg[0].shape[1]
n = 720
# initialization 
D = image_input
B = np.zeros((image_input.shape[0], image_input.shape[1]))
B_hat = np.ones((480, 720))
E = np.zeros((image_input.shape[0], image_input.shape[1]))
epsilon = np.zeros((image_input.shape[0], image_input.shape[1]))
p1 = 6
p2 = 8
S_hat = np.ones((480, 720))
alpha = 0.1
beta = 0.1 
tau = np.zeros((1, p1))
increased_tau = np.ones((1, p1))
K = np.sqrt(n)
gamma = beta
eta1 = 1/np.sqrt(2)
eta2 = 0.5


    
def decolor(D, tau, B_hat, S_hat, rank, alpha):
    # ord is 2 or fro 
    tau_hat = np.ones((1, p1*n))
    delta_hat = 0.5
    increased_tau = np.ones((1, p1*n))
    iter_num = 0
    beta = 0.1
    while iter_num < 25:
        print(iter_num)
        # to-do: the wls problem is to get the increased_tau
        # should be two ways to calculate complementary projection
        projection_inner = dot_operator(D, tau) - B_hat
        sub_projection = orthogonal_projection_try(dot_operator(D, tau) - B_hat, S_hat)
        com_projection = projection_inner - sub_projection
        increased_tau = wls_subproblem(tf.norm(com_projection, ord=2) ** 2, increased_tau)
        tau_hat = tau_hat + increased_tau
        if get_rank(B_hat) <= K:
            alpha = eta1 * alpha
            # size of B_hat is(480, 720)
            jacobian_pro = 
            B_hat = singular_threshold(com_projection(I, dot_operator(D, tau) - jacobian_pro, S_hat) + orthogonal_projection_try(B_hat, S_hat), rank, alpha)
        iter_num += 1
        delta_est = estimate_delta(D, B_hat)
        beta = np.max((eta2 * beta, 4.5 * ((delta_hat)**2)))
        # todo: the index of S is changable? 
        S_hat = (beta - 0.5 * (np.subtract(dot_operator(D, tau), B_hat) ** 2)) * S + gamma * tf.norm(np.matmul(A, np.transpose(S[0])), ord=1)
    
    return B_hat, S_hat, tau_hat
       

SyntaxError: invalid syntax (<ipython-input-1-558ca10b0989>, line 47)

In [2]:
orthogonal_projection_try(dot_operator(D, tau) - B_hat, S_hat).shape

NameError: name 'orthogonal_projection_try' is not defined

In [199]:
decolor(D, tau, B_hat, S_hat, rank, alpha)

0
(720, 480)
(720, 480)
(720, 480)


NameError: name 'jacobian_pro' is not defined

In [ ]:
torch.tensor((np.dot(B[4].reshape((1, 720)), np.transpose(S_hat))/np.tensordot(S_hat, S_hat)) * S_hat).shape
print(len(orthogonal_projection(B_hat, S_hat)))